# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *
import tomli

In [ ]:
#| hide
def check_settings(file):
    match file:
        case {
            "source": {"path": str(), "filename_geometry": str(), "filename_landmarks": str()},
            "target": {"path": str(),"filename_landmarks": str()},
        }:
            pass
        case _:
            raise ValueError(f"invalid configuration: {file}")

In [ ]:
#| hide
def read_config_file():
    with open("Config.toml", "rb") as f:
        config = tomli.load(f)
        check_settings(config)
        return config
        

In [ ]:
#| export
def morphing():
    "Apply morphing to a source template"
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()